### Checking the accuracy of a manufacturing process

This code flags products whose dimensions fall outside of the acceptable control limits (UCL and LCL) for each operator. 

In [ ]:
SELECT *
FROM manufacturing_parts;

In [ ]:
-- Checking the total numbers of parts and their average dimensions for each operator
SELECT
	operator,
	COUNT(DISTINCT item_no) AS number_of_items,
	AVG(length) AS avg_length, 
	AVG(width) AS avg_width,
	AVG(height) AS avg_height
FROM manufacturing_parts
GROUP BY operator
ORDER BY operator;

In [ ]:
-- Using a window function to find average height and standard deviation of the variable for the 5 preceding rows
WITH height_stats AS (
    SELECT 
        operator,
        item_no,
        height,
        AVG(height) OVER (
            PARTITION BY operator 
            ORDER BY item_no 
            ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
        ) AS avg_height,
        STDDEV(height) OVER (
            PARTITION BY operator 
            ORDER BY item_no 
            ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
        ) AS stddev_height,
	   ROW_NUMBER() OVER (PARTITION BY operator ORDER BY item_no) AS row_number
    FROM 
        manufacturing_parts

),
-- Flag items that do are not within the acceptable limits
limit_table AS (
	SELECT 
		*, 
		avg_height + 3 * (stddev_height/SQRT(5)) AS ucl,
		avg_height - 3 * (stddev_height/SQRT(5)) AS lcl,
		CASE 
			WHEN height <= avg_height + 3 * (stddev_height/SQRT(5)) AND height >= avg_height - 3 * (stddev_height/SQRT(5)) THEN FALSE
			ELSE TRUE
		END AS alert
	FROM 
		height_stats
	WHERE 
		row_number >=5
)
SELECT operator, row_number, height, avg_height, stddev_height, ucl, lcl, alert
FROM limit_table;